In [1]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import librosa
import cv2

2023-09-30 13:02:30.183335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
spectrogram_folder = 'spectograms'


img_width, img_height = 150, 150
batch_size = 32  


datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,   
    rotation_range=20,      
    width_shift_range=0.2, 
    height_shift_range=0.2,
    horizontal_flip=True
)


train_generator = datagen.flow_from_directory(
    spectrogram_folder,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


validation_generator = datagen.flow_from_directory(
    spectrogram_folder,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])


optimizer = Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


model.fit(train_generator, epochs=50, validation_data=validation_generator)


y_pred = np.argmax(model.predict(validation_generator), axis=-1)


print(confusion_matrix(validation_generator.classes, y_pred))
print(classification_report(validation_generator.classes, y_pred, target_names=train_generator.class_indices.keys()))


Found 360 images belonging to 5 classes.
Found 89 images belonging to 5 classes.


2023-09-30 13:02:38.261967: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
12/12 [==============================] - 19s 2s/step - loss: 1.9943 - accuracy: 0.6889 - val_loss: 1.4337 - val_accuracy: 0.8427
Epoch 2/50
12/12 [==============================] - 17s 1s/step - loss: 2.7773 - accuracy: 0.7361 - val_loss: 2.4547 - val_accuracy: 0.0449
Epoch 3/50
12/12 [==============================] - 17s 1s/step - loss: 2.4482 - accuracy: 0.6694 - val_loss: 3.9078 - val_accuracy: 0.0337
Epoch 4/50
12/12 [==============================] - 17s 1s/step - loss: 2.2561 - accuracy: 0.6389 - val_loss: 4.7490 - val_accuracy: 0.0337
Epoch 5/50
12/12 [==============================] - 17s 1s/step - loss: 1.6797 - accuracy: 0.7361 - val_loss: 5.3228 - val_accuracy: 0.0225
Epoch 6/50
12/12 [==============================] - 18s 1s/step - loss: 1.7834 - accuracy: 0.6833 - val_loss: 5.6478 - val_accuracy: 0.0337
Epoch 7/50
12/12 [==============================] - 17s 1s/step - loss: 1.2605 - accuracy: 0.6944 - val_loss: 5.8006 - val_accuracy: 0.0337
Epoch 8/50
12/12 [==

/Users/buga/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/buga/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/buga/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

In [3]:
class_names = train_generator.class_indices.keys()
print(f"Class names: {list(class_names)}")

Class names: ['belly_pain', 'burping', 'discomfort', 'hungry', 'tired']


In [4]:
audio_file = 'pred/bely_pain/C421C6FE-DFEE-4080-8AEA-848E7CE4756B-1430548333-1.0-f-26-bp.wav'
audio, sr = librosa.load(audio_file, sr=None)


spectrogram = np.abs(librosa.stft(audio))
spectrogram_db = librosa.amplitude_to_db(spectrogram, ref=np.max)


spectrogram_resized = cv2.resize(spectrogram_db, (150, 150))  
spectrogram_rgb = np.stack([spectrogram_resized]*3, axis=-1) 

spectrogram_ready = np.expand_dims(spectrogram_rgb, axis=0)


spectrogram_ready /= 255.0

# Предсказание
prediction = model.predict(spectrogram_ready)
predicted_class = np.argmax(prediction, axis=-1)

print(f"Predicted class: {predicted_class}")

1/1 [==============================] - 0s 128ms/step
Predicted class: [3]
